In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          79G   28G   52G  35% /
tmpfs            64M     0   64M   0% /dev
shm              41G     0   41G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
/dev/sda1       106G   82G   24G  78% /opt/bin/.nvidia
tmpfs            42G   52K   42G   1% /var/colab
tmpfs            42G     0   42G   0% /proc/acpi
tmpfs            42G     0   42G   0% /proc/scsi
tmpfs            42G     0   42G   0% /sys/firmware
drive            15G  4.3G   11G  29% /content/drive


In [ ]:
!pip install datasets
!pip install transformers datasets torch
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
# drive.mount('/content/drive')

# Load the dataset
file_path = '/content/drive/My Drive/cleaned_train.csv'
df = pd.read_csv(file_path)

# Path to your validation dataset
val_path = '/content/drive/My Drive/validation.csv'

# Load the validation dataset
eval_df = pd.read_csv(val_path)

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments

# Loading the GPT-2 fast tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", use_fast=True)
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

# Converted the DataFrame to a Dataset
processed_dataset = Dataset.from_pandas(df)
eval_dataset = Dataset.from_pandas(eval_df)

In [ ]:
# Set eos_token as the padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Define the sliding window tokenization function
def tokenize_function_sliding_window(examples, max_length=512, stride=128):
    tokens = tokenizer(
        examples['article'],
        truncation=False,
        padding=False,
        return_overflowing_tokens=True,
        max_length=max_length,
        stride=stride
    )
    input_ids = []
    attention_mask = []
    labels = []
    for i in range(len(tokens['input_ids'])):
        input_ids.append(tokens['input_ids'][i][:max_length])
        attention_mask.append(tokens['attention_mask'][i][:max_length])

        # Padding manually
        padding_length = max_length - len(input_ids[-1])
        if padding_length > 0:
            input_ids[-1] = input_ids[-1] + [tokenizer.pad_token_id] * padding_length
            attention_mask[-1] = attention_mask[-1] + [0] * padding_length

        labels.append(input_ids[-1])

    return {'input_ids': input_ids, 'attention_mask': attention_mask,  'labels': labels}


# Applying the sliding window tokenization function to the dataset
tokenized_dataset = processed_dataset.map(tokenize_function_sliding_window, batched=True)

# Set the format for PyTorch tensors
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Apply the sliding window tokenization function to the validation dataset
tokenized_eval_dataset = eval_dataset.map(tokenize_function_sliding_window, batched=True)

# Set the format for PyTorch tensors
tokenized_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/115777 [00:00<?, ? examples/s]

Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)

In [ ]:
from transformers import TrainingArguments, GPT2LMHeadModel, Trainer

# Set the directory to save the model
output_dir = '/content/drive/My Drive/gpt2-results'

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=2  # Gradient accumulation
)

model = GPT2LMHeadModel.from_pretrained("gpt2")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,2.813800,2.697340
2,2.628100,2.624141


TrainOutput(global_step=21708, training_loss=2.768324060512112, metrics={'train_runtime': 3756.4415, 'train_samples_per_second': 92.463, 'train_steps_per_second': 5.779, 'total_flos': 9.0750380802048e+16, 'train_loss': 2.768324060512112, 'epoch': 2.999792717473917})

In [ ]:
# Save the model and tokenizer
model.save_pretrained("/content/drive/My Drive/gpt2-results")
tokenizer.save_pretrained("/content/drive/My Drive/gpt2-results")

('/content/drive/My Drive/gpt2-results/tokenizer_config.json',
 '/content/drive/My Drive/gpt2-results/special_tokens_map.json',
 '/content/drive/My Drive/gpt2-results/vocab.json',
 '/content/drive/My Drive/gpt2-results/merges.txt',
 '/content/drive/My Drive/gpt2-results/added_tokens.json',
 '/content/drive/My Drive/gpt2-results/tokenizer.json')

In [ ]:
from huggingface_hub import HfApi, HfFolder

# Ensure you are logged in
from huggingface_hub import notebook_login
notebook_login()

# Set the repository ID with your username
username = "keerthi-balaji"  # Replace with your Hugging Face username
repo_id = f"{username}/finetuned-gpt2-smart-summary"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/keerthi-balaji/finetuned-gpt2-smart-summary/commit/69e9688abd24caf0f17ff027c97d5624ba03587a', commit_message='Upload tokenizer', commit_description='', oid='69e9688abd24caf0f17ff027c97d5624ba03587a', pr_url=None, pr_revision=None, pr_num=None)